In [44]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from math import nan, ceil
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    page = BeautifulSoup(req.text,"html.parser")
    return page

def get_jobs(page):
    jobs = []
    for a in page.find_all("a",class_="vacancy-item"):
        try:
            link = a['href']
            title = a.find("h2").get_text().strip()
            company = a.find("div",class_="vacancy-item__name").get_text().strip()
            city = a.find("div",class_="vacancy-item__location").get_text().strip()
            pos_open = a.find("div",class_="vacancy-item__info").find("strong").get_text().strip()
            desc = a.find("div",class_="vacancy-item__description").get_text().strip()
            job = {
                "Link": link,
                "Title": title,
                "Company": company,
                "City": city,
                "Position Open": pos_open,
                "Description": desc
            }
            jobs.append(job)
        except: pass
    return jobs

In [8]:
# url = "https://robin.jobs/job-offers-abroad"
# page = render_html(url)

In [ ]:
# find max page
# total = int(page.find("ul",class_="h-menu side-menu h-clearfix").find("li").find("span").get_text())
# max_page = ceil(total/6)

In [ ]:
# url = "https://robin.jobs/wp-admin/admin-ajax.php?action=getVacancyArchive&page=2&firstArchivePostId=false"
# req = requests.get(url,headers=headers)

In [ ]:
# jobs = get_jobs(page)

In [45]:
# scrape first page
url = "https://robin.jobs/job-offers-abroad"
page = render_html(url)
jobs = get_jobs(page)

# iterate and scrape the next page
i = 2
while True:
    url = f"https://robin.jobs/wp-admin/admin-ajax.php?action=getVacancyArchive&page={i}&firstArchivePostId=false"
    req = requests.get(url,headers=headers)
    data = req.json()
    result = data['firstArchivePostId']
    page = BeautifulSoup(data['postsHTML'],"html.parser")
    new_jobs = get_jobs(page)
    jobs.extend(new_jobs)
    if result != False:
        break
    else:
        i += 1

In [46]:
import pandas as pd

df = pd.DataFrame(jobs)
df.to_excel("sbaflex.xlsx",index=False,encoding="utf-8")